<a href="https://colab.research.google.com/github/duper203/RAG_Techniques_with_upstage/blob/main/upstage/11_Semantic_Chunking_for_Document_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Chunking for Document Processing






## Key Components

1. PDF processing and text extraction
2. Semantic chunking using LangChain's SemanticChunker
3. Vector store creation using FAISS and Upstage embeddings
4. Retriever setup for querying the processed documents





## Method Details

1. Document Preprocessing
The PDF is read and converted to a string using a custom read_pdf_to_string function.

2. Semantic Chunking

  1) Utilizes LangChain's SemanticChunker with Upstage embeddings

  2) Three breakpoint types are available:
  * 'percentile': Splits at differences greater than the X percentile.
  * 'standard_deviation': Splits at differences greater than X standard deviations.
  * 'interquartile': Uses the interquartile distance to determine split points.

3. Vector Store Creation


In [ ]:
! pip3 install -qU langchain-upstage langchain-community pypdf faiss-cpu langchain_experimental

In [2]:
import os
from google.colab import userdata

os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")

## Define document(s) path & Read PDf to string

In [3]:
path = "data/Understanding_Climate_Change.pdf"

In [5]:
import fitz
def read_pdf_to_string(path):
    """
    Read a PDF document from the specified path and return its content as a string.

    Args:
        path (str): The file path to the PDF document.

    Returns:
        str: The concatenated text content of all pages in the PDF document.

    The function uses the 'fitz' library (PyMuPDF) to open the PDF document, iterate over each page,
    extract the text content from each page, and append it to a single string.
    """
    # Open the PDF document located at the specified path
    doc = fitz.open(path)
    content = ""
    # Iterate over each page in the document
    for page_num in range(len(doc)):
        # Get the current page
        page = doc[page_num]
        # Extract the text content from the current page and append it to the content string
        content += page.get_text()
    return content

In [6]:
content = read_pdf_to_string(path)

## Breakpoint types:

* 'percentile': all differences between sentences are calculated, and then any difference greater than the X percentile is split.
* 'standard_deviation': any difference greater than X standard deviations is split.
* 'interquartile': the interquartile distance is used to split chunks.

In [9]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_upstage import UpstageEmbeddings

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

text_splitter = SemanticChunker(embeddings, breakpoint_threshold_type='percentile', breakpoint_threshold_amount=90) # chose which embeddings and breakpoint type and threshold to use

## Split original text to semantic chunks

In [10]:
docs = text_splitter.create_documents([content])

## Create vector store and retriever

In [11]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(docs, embeddings)
chunks_query_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

## Test the retriever

`retrieve_context_per_question`
`show_context`
from helper_function.py

In [14]:
test_query = "What is the main cause of climate change?"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)

<ipython-input-12-0e780d2d1984>:15: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = chunks_query_retriever.get_relevant_documents(question)


Context 1:
The Intergovernmental Panel on Climate Change (IPCC) has 
documented these changes extensively. Ice core samples, tree rings, and ocean sediments 
provide a historical record that scientists use to understand past climate conditions and 
predict future trends. The evidence overwhelmingly shows that recent changes are primarily 
driven by human activities, particularly the emission of greenhouse gases. Chapter 2: Causes of Climate Change 
Greenhouse Gases 
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous 
oxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is essential 
for life on Earth, as it keeps the planet warm enough to support life. However, human 
activities have intensified this natural process, leading to a warmer climate. Fossil Fuels 
Burning fossil fuels for energy releases large amounts of CO2. This includes co